---
# Projeto 2 - Ciência dos Dados

Nome: Daniel Pucciariello

Nome: Stephanie Liu

A proposta desse trabalho é avaliar se um tweet sobre o tema "Pastel" foi feito por qual tipo de usuário: um usuário que gosta de Pastel e teve uma boa experiência decorrente do fato dele ter comido o alimento ou somente demonstra gostar desse (classiicação 2), um usuário que desgoste de pastel ou teve uma má experiência ao se alimentar dele (classificação 1) ou um usuário cujo emprego da palavra pastel tenha sido utilizada para expressar um sentimento que não se relacione com o alimento de forma direta (irrelevante foi classificado como 0).



___
# Classificador automático de sentimento
## Preparando o ambiente no jupyter:



---
<font color=red, size="5"> __Importações das bibliotecas__
    
---

In [4]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji
!pip install remove_urls


In [5]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import string
import functools
import operator
import re
import emoji


---
<font color=red, size="5"> __Criando as funções para limpeza do código e separação dos emojis__
    
---

In [6]:

def split_emoji(em):
    em_split_emoji = emoji.get_emoji_regexp().split(em)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    
    return em_split

In [7]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;/]+' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)    
    
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = re.sub(r"http\S+", "", text_subbed)
    
    text_subbed=text_subbed.replace("\n", " ")
    text_subbed=text_subbed.replace("…", " ")
    text_subbed=text_subbed.replace("@", " ")
    text_subbed=text_subbed.replace("rt", "")
    text_subbed=text_subbed.replace("..............", "")
    text_subbed=text_subbed.replace("  ", " ")
    text_subbed=text_subbed.replace("   ", " ")
    text_subbed=text_subbed.replace("    ", " ")
    text_subbed=text_subbed.replace("     ", " ")
    text_subbed=text_subbed.replace("      ", " ")


    
    return text_subbed

In [8]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

#Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

---

In [9]:
data_treinamento = pd.read_excel("Excel_pastel.xlsx",sheet_name="Treinamento")

In [10]:
gostam_raw = " ".join(data_treinamento[data_treinamento.Classe==2].Treinamento)
ngostam_raw = " ".join(data_treinamento[data_treinamento.Classe==1].Treinamento)
irrelevantes_raw = " ".join(data_treinamento[data_treinamento.Classe==0].Treinamento)

In [11]:
irrelevantes = cleanup(irrelevantes_raw)
#irrelevantes
# -------- Variaveis que gostam 
gostam_limpo = cleanup(gostam_raw).lower()
tabela_absoluta_gostam = pd.Series(split_emoji(gostam_limpo)).value_counts(False)
palavras_gostam = split_emoji(gostam_limpo)
# ------- Variaveis que não gostam
ngostam_limpo = cleanup(ngostam_raw).lower()
tabela_absoluta_ngostam = pd.Series(split_emoji(ngostam_limpo)).value_counts(False)
palavras_ngostam = split_emoji(ngostam_limpo)

# ------- Variaveis Irrelevantes
irrelevantes_limpo = cleanup(irrelevantes_raw).lower()
tabela_absoluta_irrelevantes = pd.Series(split_emoji(irrelevantes_limpo)).value_counts(False)
palavras_irrelevantes = split_emoji(irrelevantes_limpo)



In [12]:
tudo=irrelevantes_raw + gostam_raw +ngostam_raw
tudo=cleanup(tudo).lower()
tudo_absoluto=pd.Series(split_emoji(tudo)).value_counts(False)


In [13]:
data_teste = pd.read_excel("Excel_pastel.xlsx",sheet_name="Teste")

In [14]:
tudo_raw = tudo.split()
tudo_sem_repeticao = set(tudo_raw)

In [34]:
# ------- Listas Vazias que irão receber valores -----------
lista=[]
teste_split_emoji=[]

# ------- CONTANDO AS VARIAVEIS -----------
count_palavras_gostam = len(palavras_gostam) #contagem das palavras classificadas como gosta
count_palavras_ngostam = len(palavras_ngostam) #contagem das palavras classificadas como não gosta
count_palavras_irrelevantes = len(palavras_irrelevantes) #contagem das palavras irrelevantes
d = len(tudo_sem_repeticao) #numero de palavras sem repetição

# ------- Naive-Bayes -----------
for i in data_teste.Teste:
    a="".join(i)
    b = split_emoji(cleanup(a).lower())
    teste_split_emoji.append(b)
    
for y in teste_split_emoji:
    gosta = 1
    ngosta = 1
    irrelevante =1
    for x in y:
        prob_g = 1
        prob_n = 1
        prob_i = 1
        if x not in tabela_absoluta_gostam:
            prob_g= 1/(count_palavras_gostam+d)
            
        elif x in tabela_absoluta_gostam:
            prob_g=((tabela_absoluta_gostam[x]+1)/(count_palavras_gostam+d))
            
        if x not in tabela_absoluta_irrelevantes:
            prob_i= 1/(count_palavras_irrelevantes+d)
            
        elif x in tabela_absoluta_irrelevantes:
            prob_i=((tabela_absoluta_irrelevantes[x]+1)/(count_palavras_irrelevantes+d))
        
        if x not in tabela_absoluta_ngostam:
            prob_n= 1/(count_palavras_ngostam+d)
            
        elif x in tabela_absoluta_ngostam:
            prob_n=((tabela_absoluta_ngostam[x]+1)/(count_palavras_ngostam+d))
        
        gosta*=prob_g
        ngosta*=prob_n
        irrelevante*=prob_i
        
    if gosta>ngosta and gosta >irrelevante:
        valor=2
    elif gosta<ngosta and ngosta>irrelevante:
        valor=1
    elif irrelevante>gosta and irrelevante>ngosta:
        valor=0
    lista.append(valor)

data_teste.Classe = lista #coluna do data_frame recebe a lista

data_teste.to_excel("Planilha_tweets_classificados_por_probabilidade.xlsx") #criaria o excel


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [16]:
errou1=len(data_teste.loc[(data_teste.Classe==2)&(data_teste.Modelo<=1)])
errou2=len(data_teste.loc[(data_teste.Classe==1)&(data_teste.Modelo==0)])
errou3=len(data_teste.loc[(data_teste.Classe==0)&(data_teste.Modelo>=1)])
errou4=len(data_teste.loc[(data_teste.Classe==1)&(data_teste.Modelo==2)])

total_errou = errou1 + errou2 + errou3 + errou4
acuracia = 1 - (total_errou/(len(data_teste)))
acuracia

0.7340153452685422

In [17]:
# true_positive1 = len(data_teste.loc[(data_teste.Classe==2)&(data_teste.Modelo==2)])/(len(data_teste))
# true_negative1 = len(data_teste.loc[(data_teste.Classe<2)&(data_teste.Modelo<2)])/(len(data_teste))

# false_positive1 = len(data_teste.loc[(data_teste.Classe==2)&(data_teste.Modelo<2)])/(len(data_teste))
# false_negative1 = len(data_teste.loc[(data_teste.Classe<2)&(data_teste.Modelo==2)])/(len(data_teste))

In [30]:
true_positive = len(data_teste.loc[(data_teste.Classe==2)&(data_teste.Modelo==2)])/(len(data_teste))*100
true_negative = len(data_teste.loc[(data_teste.Classe==1)&(data_teste.Modelo==1)])/(len(data_teste))*100
true_irrelevant = len(data_teste.loc[(data_teste.Classe==0)&(data_teste.Modelo==0)])/(len(data_teste))*100

false_positive = len(data_teste.loc[(data_teste.Classe==2)&(data_teste.Modelo<2)])/(len(data_teste))*100
false_negative = len(data_teste.loc[(data_teste.Classe==1)&(data_teste.Modelo!=1)])/(len(data_teste))*100
false_irrelevant = len(data_teste.loc[(data_teste.Classe==0)&(data_teste.Modelo!=0)])/(len(data_teste))*100

print("Porcentagem de verdadeiros positivos(mensagens que forem corretamente classificadas como 2):{0}".format(true_positive)) 
print("Porcentagem de verdadeiros negativos(mensagens que foram corretamente classificadas como 1):{0}".format(true_negative))
print("Porcentagem de verdadeiros negativos(mensagens que foram corretamente classificadas como 0):{0}".format(true_irrelevant))

print("Porcentagem de falso positivo (mensagens que foram classificadas como diferente de 2):{0}".format(false_positive)) 
print("Porcentagem de falso negativos (mensagens que foram classificadas como diferente de 1):{0}".format(false_negative))
print("Porcentagem de falso irrelevante (mensagens que foram classificadas como diferente de 0):{0}".format(false_irrelevant))

Porcentagem de verdadeiros positivos(mensagens que forem corretamente classificadas como 2):59.846547314578004
Porcentagem de verdadeiros negativos(mensagens que foram corretamente classificadas como 1):0.7672634271099744
Porcentagem de verdadeiros negativos(mensagens que foram corretamente classificadas como 0):12.787723785166241
Porcentagem de falso positivo (mensagens que foram classificadas como diferente de 2):12.787723785166241
Porcentagem de falso negativos (mensagens que foram classificadas como diferente de 1):5.626598465473146
Porcentagem de falso irrelevante (mensagens que foram classificadas como diferente de 0):8.184143222506394


___
### Concluindo

O Projeto tem como base o Teorema de Bayes, mais especificamente Classificador Naive-Bayes. Este assume que os elementos probabilísticos que irão compor um evento, cuja classificação irá ser realizada são independentes entre si. Na prática, isso significa que o cálculo da probabilidade final de acontecimento de determinado evento pode ser simplificada pela multiplicação da probabilidade de cada um dos elementos que o compõe. Aplicações plausíveis e utilizadas frequentemente são os classificadores de SPAM de emails e classificadores de assunto em um conjuntos de notícias, por exemplo. Contudo, neste projeto, usaremos o teorema para classificar tweets.

Primeiramente, foram coletados tweets 499 tweets não repetidos de acordo com 3 requisitos, de modo a criar uma base de treinamento após estes terem sua classificação realizada.. A partir disso, foi calculada a probabilidade de cada tweet ser de uma categoria ou de outra. Para tal cálculo foi utilizado uma técnica conhecida como Laplace Smoothing aliada ao conceito de classificador Naive-Bayes e dos dados anteriormente obtidos pela classificação manualmente feita.

Devido a grande quantidade de retweets, foi necessário coletar novamente os tweets para serem classificados. Contudo, neste mesmo dia foi o lançamento do novo iPhone, tendo assim comentários imprevistos, em relação a cor pastel dos novos produtos. Isso ocasionou uma interferência nos cálculos da probabilidade, devido a quantidades desproporcionais na base de treinamento e na de teste, levando uma acurácia menor do que prevista. Além disso, é necessário frisar que a quantidade de tweets classificados como negativos é significativamente baixa, o que fez com que o índice de verdadeiros negativos (classificados como 1) fosse menor do que 1%. Apesar dos imprevistos da cor pastel e a baixa quantidade de dados classificados na base de treinamento como 1, a acurácia da classificação chegou a ser maior do que 70%, o que provavelmente se relaciona com a grande aceitação de pastel na sociedade brasileira.

Para futuras interações do projeto seria ideal aumentar os tweets da base de treinamento e classificá-las a mão de modo que estes tweets abordassem diferentes assuntos relacionados a pastel, tanto a cor quanto a comida. Além de não ter sido abordado a cor pastel, há outras possíveis influências, como as figuras de linguagem que poderá estar presente no texto, como eufemismo, metáfora, ironia, e entre outros. Para tais situações deve-se implementar um mecanismo que transforma os verbos em seus respectivos infinitivos, de maneira a melhorar a acurácia. Este processo é conhecido como stemming e lemmatization e pode ser empregado utilizando bibliotecas do python como o NTLK (baseado no link da primeira referência).

Outro fator importante é que não se deve usar o próprio classificador para gerar mais amostras de treinamento, pois apesar de haver uma comparação de uma probabilidade de ser cada classificação, não há a certeza que tal resultado está certo, logo é inviável concluir que está correto em todos os momentos. Em outras palavras, caso seja utilizado o próprio classificador com a finalidade de gerar mais amostras de treinamento, estaríamos supondo que o nosso classificador está sempre certo, resultando-se em uma acurácia cada vez menor.

Por fim, é necessário ressaltar que o próprio Teorema de Bayes é uma grande limitação quando se trata de classificação de praticamente qualquer coisa. No caso de tweets isso não deixa de ser uma exceção, visto que os elementos que compõem as frases (palavras) claramente possuem uma relação entre elas, de maneira que uma irá influenciar na presença das outras palavras e o todo irá influenciar no sentido da frase (nesse caso, um tweet).


# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**